In [2]:
import numpy as np
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split

os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
try:
# For Windows
    file_path = 'D:\\Matlab_Drive\\Data\\WIFI\\180_100_DCout'
    file_list = os.listdir(file_path)
except:
# For Linux
    file_path = '/home/herokwon/data/WIFI/180_100_DCout'
    file_list = os.listdir(file_path)

file_list_data = [os.path.join(file_path,s) for s in file_list if not "_idx.npy" in s]
file_list_idx = [os.path.join(file_path,s) for s in file_list if "_idx.npy" in s]

#x_train,x_test,y_train,y_test = train_test_split(file_list_data,file_list_idx,test_size=0.2,random_state=33)
#data_train = (x_train,y_train)
#data_test = (x_test,y_test)

def gen_readnpy(files_data,train_size,test_size):
    for i in range(len(files_data[0])):
        data_read = np.load(files_data[0][i]).astype('float32')
        label_read = np.load(files_data[1][i]).astype('int32')

        x_train,x_test,y_train,y_test = train_test_split(data_read,label_read,test_size=0.2,random_state=33)
        minibatch_num = int(1000 / (train_size + test_size))
        
        for j in range(minibatch_num):
            train_x_give = x_train[train_size*j:train_size*(j+1)]
            train_y_give = y_train[train_size*j:train_size*(j+1)]
            test_x_give = x_test[test_size*j:test_size*(j+1)]
            test_y_give = y_test[test_size*j:test_size*(j+1)]
            yield train_x_give,train_y_give,test_x_give,test_y_give
            #yield train_x_give,train_y_give,train_x_give[0:20],train_y_give[0:20]

#iter = gen_readnpy((file_list_data,file_list_idx))
#iter_test = gen_readnpy(data_test)

tf.reset_default_graph()
## Network
learning_rate = 0.1
train_batch_size = 80
test_batch_size = 20

n_input = 6*30
n_step  = 500
n_class = 110
total_batch = int(8000 / (train_batch_size+test_batch_size))
n_hidden = 128

iter = gen_readnpy((file_list_data,file_list_idx),train_batch_size,test_batch_size)

X = tf.placeholder(tf.float32,[None,n_step,n_input])
Y = tf.placeholder(tf.int32,[None,n_class])

W = tf.Variable(tf.random_normal([n_hidden,n_class]))
b = tf.Variable(tf.random_normal([n_class]))

cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

outputs, states = tf.nn.dynamic_rnn(cell,X,dtype=tf.float32)

outputs = tf.transpose(outputs,[1,0,2])
outputs = outputs[-1]

model = tf.matmul(outputs,W) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=model,labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
# Accuracy
is_correct = tf.equal(tf.argmax(model,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))


In [3]:
sess = tf.Session()

In [4]:
sess.run(tf.global_variables_initializer())

In [5]:
batch_xs,batch_ys,batch_xst,batch_yst = next(iter)

In [6]:
batch_xs.shape

(80, 6, 30, 500)

In [7]:
batch_ys.shape

(80, 1)

In [8]:
            batch_xs = batch_xs.reshape((train_batch_size,n_input,n_step))
            batch_xs = np.transpose(batch_xs,[0,2,1])
            batch_ys = np.eye(n_class)[batch_ys].squeeze(1)

In [9]:
batch_xs.shape

(80, 500, 180)

In [10]:
sess.run([optimizer,cost],
            feed_dict = {X:batch_xs,Y:batch_ys})

[None, 7.2316437]

In [50]:
sess.run(tf.argmax(model,1),feed_dict = {X:batch_xs,Y:batch_ys})

array([ 52,  93,  11,  96,  52,  89,  96,  49,  11,  32,  11,  11,  75,
        25,  75,   6,  68, 101,  15,  11,  52,  11,  54,  68,  99,  11,
        97,  32,  27,  75,  11,  11,  11,  40,  40,  77,  10,  76,  52,
        11, 101,  52,  40,  11,  11,  89,  60,  48,  11,  11,  97,  52,
        89,  47,  11,  48,  11,  95, 108,  89,  12,  24,  52,  11,  49,
        40,  52,  11,  75,  11,  48,  40,  52,  15,  80,  86,  99,  52,
        11,  11], dtype=int64)

In [51]:
sess.run(tf.argmax(model,0),feed_dict = {X:batch_xs,Y:batch_ys})

array([40, 64, 38, 17,  9,  6, 15, 13, 26, 57, 36, 78, 64, 23, 18, 18, 39,
        6, 38, 53,  9, 57, 53, 58, 61, 13,  2, 17, 35,  6, 17,  8, 27, 41,
       61,  7, 56,  1, 36,  7, 38, 64, 36,  3,  7, 13, 53, 53,  2, 64,  0,
       64, 41,  2, 27, 17, 70, 18, 18, 67, 46, 27, 27,  6,  9, 18, 35, 18,
       27, 18, 26, 76, 63, 18, 57, 18, 41, 35, 27,  7, 27,  6, 53, 64,  9,
       39, 75, 18,  3,  5, 35, 13, 37,  1, 33, 57,  6, 26, 18, 14, 35, 17,
       28, 27, 73, 75, 35, 27, 58, 27], dtype=int64)

In [52]:
sess.run(tf.argmax(Y,1),feed_dict = {X:batch_xs,Y:batch_ys})

array([ 23,   3,  43,  29,  47,  50,  98,   9,  19,  32,  73,  92,  75,
        45,  12,   6,  54,   3,  15,  87,  89, 100,  54,  24,  16,  65,
        97,  32,   6,  75,  66,  54,  74,  94,  76,   9,  10,  33,   2,
        19,  18,  52,  30,  25,  88,  80,  18,  81,  37,  85,   6,  50,
        50,   9,  60,  48,  97,   8,  23,  20,  12,  29,  68,  19,   1,
        49,  71,  50,  58,  76,   7,  63,  72,  15,  40,  19,  37,  89,
        11,  33], dtype=int64)

In [53]:
sess.run(tf.argmax(Y,0),feed_dict = {X:batch_xs,Y:batch_ys})

array([ 0, 64, 38,  1,  0,  0, 15, 70, 57,  7, 36, 78, 14,  0,  0, 18, 24,
        0, 40,  8, 59,  0,  0,  0, 23, 43,  0,  0,  0,  3, 42,  0,  9, 37,
        0,  0,  0, 48,  0,  0, 74,  0,  0,  2,  0, 13,  0,  4, 55, 65,  5,
        0, 41,  0, 16,  0,  0,  0, 68,  0, 54,  0,  0, 71,  0, 25, 30,  0,
       62,  0,  0, 66, 72, 10, 32, 12, 34,  0,  0,  0, 45, 47,  0,  0,  0,
       49,  0, 19, 44, 20,  0,  0, 11,  0, 33,  0,  0, 26,  6,  0, 21,  0,
        0,  0,  0,  0,  0,  0,  0,  0], dtype=int64)

In [56]:
batch_ys[1]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

In [58]:
a = sess.run(model,feed_dict = {X:batch_xs,Y:batch_ys})

In [59]:
a.shape

(80, 110)

In [60]:
a[0]

array([-4.8415294 , -0.78726184,  1.1875112 ,  1.7281266 , -3.6410387 ,
       -0.08857   , -5.9726105 , -0.1352359 ,  3.675872  ,  2.9023352 ,
       -3.4347086 ,  9.1046505 ,  0.11783755,  0.12506664, -0.72511625,
        0.02103913,  3.8430076 ,  3.5527067 ,  1.4929217 , -1.1902844 ,
        2.279029  , -5.483029  , -6.218196  ,  0.09862357,  6.494483  ,
       -1.14462   ,  2.6145265 , -4.217497  , -1.5848749 , -2.4082303 ,
        0.7926787 ,  4.7689605 , -7.883393  , -1.1275276 , -2.8279278 ,
        0.88098073, -0.3380128 , -0.76947683, -3.8957562 , -7.6777716 ,
        5.7663155 , -0.09414703,  5.027512  ,  1.104957  , -3.3626142 ,
        0.84067476, -0.57693034, -2.016649  ,  6.2038856 ,  0.6910533 ,
        8.932434  , -6.351075  ,  9.80212   ,  2.3050375 , -3.0300894 ,
       -1.9651577 ,  0.513273  ,  1.7570096 , -3.6051536 , -0.38810933,
        5.159661  , -3.654144  , -7.3313417 , -2.0973108 , -1.7042792 ,
       -2.2532399 , -4.404416  , -0.15728092,  8.746478  , -1.21

In [63]:
np.max(a[0])

9.80212

In [64]:
import numpy as np
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split

os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
try:
# For Windows
    file_path = 'D:\\Matlab_Drive\\Data\\WIFI\\180_100_DCout'
    file_list = os.listdir(file_path)
except:
# For Linux
    file_path = '/home/herokwon/data/WIFI/180_100_DCout'
    file_list = os.listdir(file_path)

file_list_data = [os.path.join(file_path,s) for s in file_list if not "_idx.npy" in s]
file_list_idx = [os.path.join(file_path,s) for s in file_list if "_idx.npy" in s]

#x_train,x_test,y_train,y_test = train_test_split(file_list_data,file_list_idx,test_size=0.2,random_state=33)
#data_train = (x_train,y_train)
#data_test = (x_test,y_test)

def gen_readnpy(files_data,train_size,test_size):
    for i in range(len(files_data[0])):
        data_read = np.load(files_data[0][i]).astype('float32')
        label_read = np.load(files_data[1][i]).astype('int32')

        x_train,x_test,y_train,y_test = train_test_split(data_read,label_read,test_size=0.2,shuffle=False)
        minibatch_num = int(1000 / (train_size + test_size))
        
        for j in range(minibatch_num):
            train_x_give = x_train[train_size*j:train_size*(j+1)]
            train_y_give = y_train[train_size*j:train_size*(j+1)]
            test_x_give = x_test[test_size*j:test_size*(j+1)]
            test_y_give = y_test[test_size*j:test_size*(j+1)]
            yield train_x_give,train_y_give,test_x_give,test_y_give
            #yield train_x_give,train_y_give,train_x_give[0:20],train_y_give[0:20]

#iter = gen_readnpy((file_list_data,file_list_idx))
#iter_test = gen_readnpy(data_test)

tf.reset_default_graph()
## Network
learning_rate = 0.1
train_batch_size = 80
test_batch_size = 20

n_input = 6*30
n_step  = 500
n_class = 110
total_batch = int(8000 / (train_batch_size+test_batch_size))
n_hidden = 128

iter = gen_readnpy((file_list_data,file_list_idx),train_batch_size,test_batch_size)

X = tf.placeholder(tf.float32,[None,n_step,n_input])
Y = tf.placeholder(tf.int32,[None,n_class])

W = tf.Variable(tf.random_normal([n_hidden,n_class]))
b = tf.Variable(tf.random_normal([n_class]))

cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

outputs, states = tf.nn.dynamic_rnn(cell,X,dtype=tf.float32)

outputs = tf.transpose(outputs,[1,0,2])
outputs = outputs[-1]

model = tf.matmul(outputs,W) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=model,labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
# Accuracy
is_correct = tf.equal(tf.argmax(model,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))

#tf.reset_default_graph()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    total_cost = 0

    for i in range(total_batch):
        try:
            batch_xs,batch_ys,batch_xst,batch_yst = next(iter)

            batch_xs = batch_xs.reshape((train_batch_size,n_input,n_step))
            batch_xs = np.transpose(batch_xs,[0,2,1])
            batch_ys = np.eye(n_class)[batch_ys].squeeze(1)
            
            _,cost_val = sess.run([optimizer,cost],
            feed_dict = {X:batch_xs,Y:batch_ys})
            
            batch_xst = batch_xst.reshape((test_batch_size,n_input,n_step))
            batch_xst = np.transpose(batch_xst,[0,2,1])
            batch_yst = np.eye(n_class)[batch_yst].squeeze(1)

            acc_t,cost_val_t = sess.run([accuracy,cost],feed_dict = {X:batch_xst,Y:batch_yst})

            # Accuracy
            #is_correct = tf.equal(tf.argmax(model,1),tf.argmax(Y,1))
            #accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))

            #acc_t = sess.run(accuracy,feed_dict = {X:batch_xst,Y:batch_yst})

            total_cost += cost_val

        except tf.errors.OutOfRangeError:
            print("End of Dataset")
        
        print('Epoch:','%04d' % (i+1),
        'Avg.cost = ','{:.3f}'.format(cost_val),'cost_t = ',cost_val_t,'Acc = ',acc_t)



Epoch: 0001 Avg.cost =  7.575 cost_t =  16.919094 Acc =  0.0
Epoch: 0002 Avg.cost =  15.529 cost_t =  13.715799 Acc =  0.0
Epoch: 0003 Avg.cost =  14.009 cost_t =  9.127572 Acc =  0.0
Epoch: 0004 Avg.cost =  15.722 cost_t =  12.983557 Acc =  0.0
Epoch: 0005 Avg.cost =  11.444 cost_t =  9.159189 Acc =  0.0
Epoch: 0006 Avg.cost =  10.864 cost_t =  10.914895 Acc =  0.0
Epoch: 0007 Avg.cost =  10.929 cost_t =  10.963301 Acc =  0.0
Epoch: 0008 Avg.cost =  8.543 cost_t =  10.161664 Acc =  0.0
Epoch: 0009 Avg.cost =  10.486 cost_t =  10.561976 Acc =  0.0
Epoch: 0010 Avg.cost =  10.531 cost_t =  10.045057 Acc =  0.0
Epoch: 0011 Avg.cost =  9.632 cost_t =  10.280188 Acc =  0.0
Epoch: 0012 Avg.cost =  8.989 cost_t =  8.461498 Acc =  0.0


KeyboardInterrupt: 

In [ ]:
1